library(ggmap)
library(mapproj)
library(plyr)
library(dplyr)
library(tidyr)
library(ggplot2)
library(gridExtra)
library(ggrepel)


data.source <- read.csv("D:/Kaggle/SouthAfricaCrimeStats_v2.csv", header = TRUE,sep = ",")
head(data.source)

data.cleaned <- data.source
colnames(data.cleaned)[5:14]
colnames(data.cleaned)[5:14]<- gsub("X","",names(data.cleaned)[5:14])
colnames(data.cleaned)[5:14]

data.cleaned <- gather(data.cleaned, year, freq, 5:14)
data.cleaned
data.cleaned$Category 
province.overview <- data.cleaned[data.cleaned$Category == "All theft not mentioned elsewhere",]
province.overview
province.overview <- aggregate(data=province.overview, freq ~ Province + year,FUN="sum")
province.overview
my.palette <- c("#5bc0eb", "#fde74c", "#9bc53d", "#e55934", "#fa7921", "#b6174b", "#205921", "#e99f4e", "#999999")

ggplot(province.overview, aes(x=year,y=freq/1000,colour=Province,group=Province))+
  geom_point(size=5)+
  geom_line(size= 1.2)+
  theme_bw()+
  theme(legend.position="top")+
  coord_trans(y="log2")+
  scale_colour_manual(values = my.palette)+
  ylab("Number of crimes in thousands")+
  geom_point(aes(x=year,y=freq/1000),colour="white",size=3)+
  ggtitle("Number of crimes in thousands per province by year with log2 y coordinate")

province.overview
province.overview <- filter(province.overview, year %in% c("2006.2007", "2014.2015"))
province.overview
province.overview <- spread(province.overview, year,freq,-province.overview$Province)
province.overview
province.overview <- mutate(province.overview,Size = `2014.2015` - `2006.2007`)
province.overview
province.overview <- mutate(province.overview, 'Crime evolution' = sign(Size))
province.overview
province.overview$`Crime evolution` <- gsub("-1","Improved", province.overview$`Crime evolution`)
province.overview
province.overview$`Crime evolution` <- gsub("1","Deteriorated", province.overview$`Crime evolution`)
province.overview
province.overview$Size <- abs(province.overview$Size)
province.overview$Size
#get geo code
province.overview$full.province <- paste(province.overview$Province, "South Africa")

latlong <- geocode(province.overview$full.province)

province.overview <- cbind(province.overview, latlong)

#get map
SA.map <- get_map("South Africa", 
                  zoom = 5, 
                  source = "stamen",
                  maptype = "watercolor") #toner

#chart
ggmap(SA.map, extent = "device") +
  geom_point(data= province.overview,
             aes(x = lon, y = lat, 
                 size=Size, 
                 colour=`Crime evolution`))+
  scale_size_continuous(range = c(4,10))+
  scale_color_manual(values = c("#cc0000","#00d27f"))+
  geom_text_repel(data= province.overview,
                  force=10,
                  aes(label=Province))+
  ggtitle("Crime number evolution between 2005-2006 and 2014-2015")
